# Active Internet Connection Visualizer

See all of your active connections (i.e. the output of the `netstat` command and have them plotted to see which areas of the world they are coming from.

## Imports

In [3]:
from dotenv import load_dotenv
import pandas as pd
import plotly.express as px

import asyncio
import ipinfo
import os
import re
import subprocess

ImportError: cannot import name 'loadenv' from 'dotenv' (/home/agupta/Documents/projects/current-connection-visualizer/venv-conn-viz/lib/python3.8/site-packages/dotenv/__init__.py)

## Get IPs of all active connections
This section accesses the local shell commands to get the external IPs of all active connections.

### Get all active connetions
Note that this uses the `netstat -n -t` command, which essentially finds all TCP connections on your computer.

In [ ]:
outputs = subprocess.run(["netstat", "-n", "-t"], capture_output=True).stdout.decode('utf-8').split('\n')
outputs[:10]

### Extract all of the IP addresses

Use the below `ip_matcher` if you only want established connections to be considered.

In [ ]:
# ip_matcher = r'tcp\W+\d\W+\d \d+.\d+.\d+.\d+:\d+\W+(?P<ip>(\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3})):\d+\W+ESTABLISHED'

Or use this `ip_matcher`, which allows `CLOSE_WAIT` and `TIME_WAIT` connections to be considered

In [ ]:
ip_matcher = r'tcp\W+\d\W+\d \d+.\d+.\d+.\d+:\d+\W+(?P<ip>(\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3})):\d+\W+'

Extract the IP addresses. Note that the loopback address `127.0.0.1` is automatically stripped out.

In [ ]:
ips = []
for line in outputs:
    m = re.match(ip_matcher, line)
    if m and m.group('ip') != '127.0.0.1':
        ips.append(m.group('ip'))
ips[:10]

## IP $\rightarrow$ location

### Login and create Handler
Get the user token. Note that it should be defined in a `.env` file in the same directory as this notebook

In [ ]:
load_env()
token = os.get_env('IPINFO_TOKEN')

And now the handler can be made

In [2]:
handler = ipinfo.getHandlerAsync(token)

NameError: name 'ipinfo' is not defined

### Get the coordinates

In [ ]:
async def get_coords(ip):
    details = await handler.getDetails(ip)
    lat, _, long = details.loc.partition(',')
    return float(lat), float(long), details.city

Use the async `get_coords` function to concurrently get all of the ip locations and put them into a pandas dataframe:

In [ ]:
results = await asyncio.gather(*map(get_coords, ips))
results_df = pd.DataFrame(results, columns=['lat', 'lon', 'city'])
results_df = results_df.groupby(['lat', 'lon', 'city']).size().reset_index(name='count')
results_df

## Graph the results

In [ ]:
fig = px.scatter_geo(results_df,
                     lat=results_df.lat,
                     lon=results_df.lon,
                     hover_name='city',
                     size='count')
fig.update_layout(title='Locations of Active Internet Connections')
fig.show()